In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

In [3]:
system_prompt = """

You are a personal chef. The user will give you a list of ingredients they have left over in their house.

Using the web search tool, search the web for recipes that can be made with the ingredients they have.

Return recipe suggestions and eventually the recipe instructions to the user, if requested.

"""

In [6]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

tool_models = [model for model in ChatNVIDIA.get_available_models() if model.supports_tools]
for elem in tool_models:
    print(elem)

id='qwen/qwen3-next-80b-a3b-instruct' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=True supports_structured_output=True supports_thinking=False thinking_prefix=None no_thinking_prefix=None thinking_param_enable=None thinking_param_disable=None base_model=None
id='openai/gpt-oss-120b' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=True supports_structured_output=True supports_thinking=False thinking_prefix=None no_thinking_prefix=None thinking_param_enable=None thinking_param_disable=None base_model=None
id='nvidia/llama-3.1-nemotron-nano-4b-v1.1' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=True supports_structured_output=False supports_thinking=True thinking_prefix='detailed thinking on' no_thinking_prefix='detailed thinking off' thinking_param_enable=None thinking_param_disable=None base_model=None
id='mistralai/mistral-nemotron' model_type='chat' client='ChatNVIDIA' endpoint=None a

In [7]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

model = ChatNVIDIA(model = "qwen/qwen3-next-80b-a3b-instruct", temperature=0.3)

agent = create_agent(
    model=model,
    tools=[web_search],
    system_prompt=system_prompt,
    checkpointer=InMemorySaver()
)

In [10]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="I have some leftover ribs and lettuce. What can I make?")]},
    config
)

print(response['messages'][-1].content)

You’ve got leftover ribs and lettuce — that’s a fantastic combo for a fresh, flavorful, and easy meal! Here are the top recipe ideas you can make:

---

### 🌯 **1. BBQ Rib Lettuce Wraps** *(Top Recommendation!)*  
A light, low-carb, and delicious way to enjoy your ribs with crisp lettuce.

**Ingredients:**
- Leftover BBQ ribs (meat pulled off the bone)
- Large butter lettuce, romaine, or iceberg leaves (for wraps)
- 1/4 cup BBQ sauce (reserve some from the ribs, or use your favorite)
- Optional toppings: thinly sliced red onion, pickled jalapeños, shredded carrot, chopped cilantro, avocado slices
- Optional sauce: sriracha mayo (mix 2 tbsp mayo + 1 tsp sriracha)

**Instructions:**
1. Gently pull the meat off the ribs and shred it into bite-sized pieces.
2. Warm the meat slightly in a skillet with a splash of BBQ sauce (about 2–3 minutes).
3. Lay out lettuce leaves on a plate — each leaf will serve as a “wrap.”
4. Spoon a few tablespoons of warm rib meat into each leaf.
5. Add your favo

In [11]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='I have some leftover chicken and rice. What can I make?', additional_kwargs={}, response_metadata={}, id='12b760ec-90b8-481b-acdf-235fe28708b2'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4f6c65352be6446896eddf2d', 'index': -1, 'type': 'function', 'function': {'name': 'web_search', 'arguments': '{"query": "recipes using leftover chicken and rice"}'}}]}, response_metadata={'role': 'assistant', 'content': None, 'reasoning_content': None, 'tool_calls': [{'id': 'call_4f6c65352be6446896eddf2d', 'index': -1, 'type': 'function', 'function': {'name': 'web_search', 'arguments': '{"query": "recipes using leftover chicken and rice"}'}}], 'token_usage': {'prompt_tokens': 212, 'total_tokens': 237, 'completion_tokens': 25, 'prompt_tokens_details': None, 'reasoning_tokens': 0}, 'finish_reason': 'tool_calls', 'model_name': 'qwen/qwen3-next-80b-a3b-instruct'}, id='lc_run--019c88de-07dc-7d40-b20b-1fa8f992b400-0', tool_calls=[{'